In [2]:
import torch.distributed as dist
import torch.multiprocessing as mp
import os

`dist.init_process_group`

``` python
def init_process(rank, size, backend='nccl'):
    os.environ['MASTER_ADDR'] = '127.0.0.1'
    os.environ['MASTER_PORT'] = '29500'
    dist.init_process_group(backend, rank, world_size=size)
```

- 分布式进程组（distributed progress group）
    - 主节点：master node；localhost就是本机；
    - MASTER_ADDR/MASTER_PORT: 设置主节点的地址及端口号，主要用于分布式的管理；
    - 哪怕是单机（单节点）多卡，也要显示设置主节点；虽然它们都在同一台机器上，但逻辑上需要一个“中心协调者”，进程间需要通过主节点完成握手
- 单机双卡，2个进程（Processes），每个进程都会调用 init_process 来初始化分布式环境；

- 进程间通信的后端：communication backend
    - NCCL: NVIDIA Collective Communication Library，仅限NVIDIA GPU
    - Gloo: Facebook自研，CPU/GPU
    - MPI

| Backend | 支持设备     | 支持通信操作                              | 支持 Float32 | 支持 Float16 | 备注 |
|---------|--------------|--------------------------------------------|--------------|--------------|------|
| MPI     | CPU, GPU     | 所有操作（All）                            | 是           | 否           | 需系统预装 MPI（如 OpenMPI） |
| GLOO    | CPU（全）<br>GPU（部分：broadcast, all-reduce） | CPU：所有<br>GPU：broadcast, all-reduce, reduce, all-gather | 是           | 是           | PyTorch 内置，适合 CPU 或调试 |
| NCCL    | GPU only     | 所有操作（All）                            | 是           | 是           | PyTorch 内置，GPU 训练首选 |

总结：
| 场景               | 推荐后端                      |
| ---------------- | ------------------------- |
| 单机/多机 **GPU 训练** | ✅ `nccl`（速度最快，支持 Float16） |
| CPU 训练或调试        | ✅ `gloo`（内置，跨平台）          |
| 超算/已有 MPI 集群     | ✅ `mpi`（需自己编译安装）          |


- 后端（backend）决定“用什么语言说话”
- 初始化方法（init_method）决定“第一次怎么互相找到对方”

---

Point-to-Point Communication

- send & recv
    - send 和 recv 是 **点对点阻塞式** 操作；
    - 只有配对的 send 与 recv 都到达后，数据才真正完成传输，程序才会继续往下走。

``` python 
def run(rank, size):
    torch.cuda.set_device(rank)
    tensor = torch.zeros(1).to(rank)
    if rank == 0:
        tensor += 1
        dist.send(tensor=tensor, dst=1)
    else:
        print('init tensor',tensor)
        dist.recv(tensor=tensor, src=0)
    print(f'Rank: {rank}, has data {tensor}')
```

输出：

``` text
init tentor tensor([0.], device='cuda:1')
Rank: 1, has data tensor([1.], device='cuda:1')
Rank: 0, has data tensor([1.], device='cuda:0')
```

---

Collective Communication

``` python
"""All-Reduce example"""
def run(rank, size):
    group = dist.new_group([0,1])
    if rank == 0:
        tensor = torch.tensor([1.,2.,3.])
    else:
        tensor = torch.tensor([4.,5.,6.])
    tensor = tensor.to(rank)
    print(f'Rank: {rank}, random tensor: {tensor}')
    dist.all_reduce(tensor, op=dist.ReduceOp.SUM, group=group)
    print(f'Rank: {rank}, has data: {tensor}')
```

输出：

``` text
Rank: 1, random tensor: tensor([4., 5., 6.], device='cuda:1')
Rank: 0, random tensor: tensor([1., 2., 3.], device='cuda:0')
Rank: 1, has data: tensor([5., 7., 9.], device='cuda:1')
Rank: 0, has data: tensor([5., 7., 9.], device='cuda:0')
```

--- 

多进程管理

``` python
if __name__ == "__main__":
    size = 2
    processes = []
    mp.set_start_method("spawn")
    for rank in range(size):
        p = mp.Process(target=init_process, args=(rank, size, run))
        p.start()
        processes.append(p)
    
    for p in processes:
        p.join()
```

- 通过`mp.set_start_method("spawn")`设置进程的启动方式为`spawn`
    - `spawn`方式会为每个子进程创建一个全新的Python解释器进程
    - `mp.get_start_method()` => `fork`（ubuntu system）
- 通过在一个循环中对所有进程调用 join() 方法，主进程会等待所有子进程执行完成后再继续执行。
    - join()主进程阻塞，等待所有子进程完成它们的任务，这是确保数据完整性和避免竞争条件的重要机制

| 参数 | 作用 |
|---|---|
| `Process` | **工厂**：生成一条**新的操作系统进程**。 |
| `target` | **订单**：告诉新进程“你一生下来就执行哪个函数”。 |

---

### 1. `mp.Process`
- 是 `multiprocessing` 的**进程类**。  
- 调用 `Process(...)` **并不开始执行**，只是**把“待办任务”打包成对象**；  
- 必须再调用 `.start()` 才会真正 fork/spawn 出一个**独立 Python 解释器进程**。

---

### 2. `target`
- 必须是**可调用对象**（函数 / 类 `__call__`）。  
- 新进程启动后，**入口就等价于**  
  ```python
  target(*args, **kwargs)
  ```
  在你给的代码里就是  
  ```python
  init_process(rank, size, run)
  ```

---

### 3. 类比
把 `Process` 想成“快递单”：  
- `target` 写**收货地址**（函数），  
- `args` 写**包裹内容**（参数），  
- `.start()` 相当于“发货”——系统立刻派一辆新车（子进程）把包裹送到目的地并执行。

---

### 4. 一句话总结
`Process` 负责**生**进程；  
`target` 负责告诉新生进程**“你活着的第一件事就是跑这个函数”**。

In [2]:
import torch
torch.zeros(1)[0]

tensor(0.)